<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/CustomerService.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install websockets openai asyncio --quiet
import asyncio
import websockets
import json

# Define la URL de conexión de la API Realtime
REALTIME_API_URL = "wss://api.openai.com/v1/realtime"

# Coloca aquí tu clave de la API de OpenAI
API_KEY = "tu-api-key"

# Función para enviar una solicitud de chat
async def connect_realtime():
    # Crea el encabezado con la autenticación
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    # Abre una conexión WebSocket
    async with websockets.connect(REALTIME_API_URL, extra_headers=headers) as websocket:
        # Primer mensaje de bienvenida
        welcome_message = {
            "model": "gpt-4o-realtime-preview",
            "messages": [
                {"role": "system", "content": "You are a customer support agent for a pharmaceutical company. Respond to questions about medications, dosages, side effects, and availability in a professional manner."},
                {"role": "user", "content": "Hello, I need help with a medication I purchased."}
            ]
        }

        # Envía el mensaje de bienvenida
        await websocket.send(json.dumps(welcome_message))

        # Espera la respuesta del modelo
        while True:
            response = await websocket.recv()
            response_data = json.loads(response)

            # Muestra la respuesta del modelo
            print(f"Response: {response_data['choices'][0]['message']['content']}")

            # Toma nueva entrada del usuario
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            # Enviar nuevo mensaje basado en el input del usuario
            new_message = {
                "model": "gpt-4o-realtime-preview",
                "messages": [
                    {"role": "user", "content": user_input}
                ]
            }
            await websocket.send(json.dumps(new_message))

# Ejecuta la conexión
asyncio.run(connect_realtime())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


RuntimeError: asyncio.run() cannot be called from a running event loop